In [ ]:
import os
from langgraph.graph import StateGraph,START,END
from src.agents.sql_agent import sql_node
from src.agents.rag_agent import rag_node
from src.agents.web_search_agent import web_search_node
from src.agents.summarize_agent import summarize
from typing import TypedDict


class State(TypedDict):
    question: str
    sql_answer: str
    internet_answer: list[dict]
    rag_answer: list
    answer: str
    pdf_paths: list[str]
    csv_paths: list[str]
    source: list[str]
    internet_flag: bool
    

# Ensure folders exist
os.makedirs("src/data/sheets", exist_ok=True)
os.makedirs("src/data/docs", exist_ok=True)

# Get only CSV and PDF file paths
csv_paths = [f for f in os.listdir("src/data/sheets") if f.endswith(".csv")]
rag_paths = [f for f in os.listdir("src/data/docs") if f.endswith(".pdf")]

# Enable web search by default
internet_flag = True

pdf_files = os.listdir("src/data/docs")
if pdf_files:
    pdf_paths = [os.path.join("src/data/docs", pdf) for pdf in pdf_files]
csv_files = os.listdir("src/data/sheets")
if csv_files:
    csv_paths = [os.path.join("src/data/sheets", csv) for csv in csv_files] if csv_files else []



graph = StateGraph(State)
graph.add_node("sql", sql_node)
graph.add_node("rag", rag_node)
graph.add_node("internet", web_search_node)
graph.add_node("summarize", summarize)


graph.add_conditional_edges(
    START,
    lambda state: (
        "sql" if state.get("csv_paths") else
        "rag" if state.get("pdf_paths") else
        "internet"
    ),
    path_map={"sql": "sql", "rag": "rag", "internet": "internet"}
)

# SQL → RAG and/or Internet → Summarize
graph.add_conditional_edges(
    "sql",
    lambda state: (
        "rag" if state.get("pdf_paths") else
        "internet" if state.get("internet_flag") else
        "summarize"
    ),
    path_map={"rag": "rag", "internet": "internet", "summarize": "summarize"}
)

# RAG → Internet or Summarize
graph.add_conditional_edges(
"rag",
lambda state: "internet" if state.get("internet_flag") else "summarize",
path_map={"internet": "internet", "summarize": "summarize"}
)
graph.add_conditional_edges(
    "internet",
    lambda state: "summarize",
    path_map={"summarize": "summarize"}
)

graph.add_edge("summarize", END)
# Compile
agent_executor = graph.compile()



In [ ]:
from IPython.display import Image, display
display(Image(agent_executor.get_graph().draw_mermaid_png()))